In [1]:
probes = {}
with open("hm450_gene_annotations.tsv") as in_handle:
    with open("hm450_gene_annotations.bed", "w") as out_handle:
        h = in_handle.next().replace("\"","").strip().split("\t")
        for line in in_handle:            
            c = line.strip().replace("\"","").split("\t")
            probes[c[0]] = c
            print >>out_handle, "\t".join(c[1:4] + [c[0],".","+"] )
          
            

In [2]:
from pybedtools import BedTool

meth = BedTool("hm450_gene_annotations.bed")
mirna = BedTool("/home/lpantano/orch/groups/seqcluster/data/annotation/hsa/hsa.gff3")

In [9]:
ann = meth.intersect(mirna,wo=True)

for line in ann:
    if line[8] == "miRNA_primary_transcript":
        precursor = line[14].split(";")[2].split("=")[1]
        probes[line[3]]=probes[line[3]] + [precursor]
    
with open("hm450_gene_annotations_w_mirna.tsv", "w") as out_handle:
    for p in probes:
        print >>out_handle, "\t".join(probes[p])
            

trying to map directly

In [91]:
from collections import defaultdict
mirbase = {}
syn = defaultdict(set)
with open("/home/lpantano/orch/groups/seqcluster/data/mirbase/miRNA.str") as in_handle:
    for line in in_handle:
        if line.startswith(">hsa"):
            c = line.strip().split()
            pre = c[0][1:]
            m1 = c[2][1:].split(":")[0]
            m2 = ""
            if len(c) > 3:
                m2 = c[3][1:].split(":")[0]
            mirbase[pre] = m1 + ";" + m2
            syn_name_base = "".join(pre.split("-")[1:3]).upper()
            if pre.find("mir-203") > -1:
                print syn_name_base.endswith("A") or syn_name_base.endswith("B") or syn_name_base.endswith("C")
            syn[syn_name_base].add(pre)
            number = pre.split("-")
            if len(number) > 3:
                syn_name = syn_name_base + "-" + number[3]
                syn[syn_name].add(pre)
                syn_name = syn_name_base + number[3]
                syn[syn_name].add(pre)
                syn_name = syn_name_base[-1] + number[3]
                syn[syn_name].add(pre)
            if syn_name_base.endswith("A") or syn_name_base.endswith("B") or syn_name_base.endswith("C"):
                syn[syn_name_base[:-1]].add(pre)
                if len(number) > 3:
                    syn_name = syn_name_base[:-1] + "-" + number[3]
                    syn[syn_name].add(pre)
                    syn_name = syn_name_base[:-1] + number[3]
                    syn[syn_name].add(pre)
            
print syn["LET7A1"]
print syn["MIR129-1"]
print syn["MIR1270"]

True
True
set(['hsa-let-7a-1'])
set(['hsa-mir-129-1'])
set(['hsa-mir-1270'])


In [94]:
with open("hm450_gene_annotations.tsv") as in_handle:
    with open("hm450_gene_annotations_mapped_mirns.tsv", "w") as out_handle:

        h = in_handle.next().replace("\"","").strip().split("\t")
        for line in in_handle:            
            c = line.strip().replace("\"","").split("\t")
            find = []
            if c[14].find("MIR") > -1:
                names = [n for n in c[14].split(";") if n.startswith("MIR")]
                new_names = []
                for n in names:
                    if n.find("LET") > -1:
                        n = n.replace("MIR","")
                    n = n.replace("HG","")
                    if n.startswith("LET") and len(n)>6:
                        for p in range(4,len(n)):
                            new_names.append(n[:4] + n[p])
                    else:
                        new_names.append(n)
                find = []
                for n in new_names:
                    
                    if n in syn:
                        find.extend(list(syn[n]))
                    elif n.endswith("-1"):
                        if n[:-2] in syn:
                            find.extend(list(syn[n[:-2]]))
                    elif n[:-1] in syn:
                        find.extend(list(syn[n[:-1]]))
                    if n=="MIR3545":
                        if "MIR203B" in syn:
                            find.extend(list(syn["MIR203B"]))
                    if not find:
                        print n
                        raise ValueError(n)

                print >>out_handle, line.strip() + "\t" + ";".join([mirbase[p] for p in find])
